In [1]:
import dataprocessing.datacolect as dc
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By

# 1. Sobre a coleta

# 2. Rotina de *scraping*

## 2.1. Abertura do *browser*

In [2]:
url = 'https://www.xbox.com/pt-br/xbox-game-pass/games?xr=shellnav'

In [3]:
webdriver = dc.Webdriver()

In [4]:
browser = webdriver.set_driver(headless=True)
browser.get(url)

## 2.2. Coleta dos *links* para as páginas dos jogos

In [5]:
scrape = dc.Scrape(browser)

In [6]:
# Ajuste da quantidade de jogos que são mostrados na página:

mais_jogos_por_pagina = scrape.get_element(
    path='//div[@data-placeholder="20 jogos por página"]',
    timeout=30
)
mais_jogos_por_pagina.click()

duzentos_jogos_por_pagina = scrape.get_element(
    path='//li[@id="unique-id-for-paglist-generated-select-menu-3"]',
    timeout=30
)
duzentos_jogos_por_pagina.click()

In [7]:
# Função para obtenção dos links para as páginas dos jogos:

def coleta_links_jogos(path: str):
    links = scrape.get_elements(path=path, timeout=20)
    links_list = list()

    for link in links:
        link = link.get_attribute('href')
        links_list.append(link)

    return links_list

def prox_pagina(path: str):
    botao_pagina = scrape.get_element(path=path, timeout=10)
    botao_pagina.click()

In [8]:
links_na_pagina = dict()

In [9]:
for iteration in range(1, 4):
    links_na_pagina[f'{iteration}'] = coleta_links_jogos(
        path='//a[@class="gameDivLink"]'
    )

    if iteration == 3:
        break
    
    prox_pagina('//li[@class="paginatenext"]//a[@class="c-glyph"]')

In [10]:
browser.quit()

## 2.3. Coleta dos dados sobre os jogos

In [11]:
dados_jogo = dict()
dados_jogos = list()

In [12]:
# for jogos in jogos_na_pagina[:10]:
#     link_jogo = jogos.get_attribute('href') # link para página do jogo.
#     browser.execute_script("window.open('', '_blank');") # abre uma nova aba do Chrome.

#     # Seleciona-se a aba aberta e o link para o jogo é aberto:
#     scrape.switch_tab(tab=1, timeout=10)
#     browser.get(link_jogo)
    
#     # Scraping dos dados sobre o jogo:
#     dados_jogo['nome_jogo'] = scrape.get_element(
#         path='//h1[@class="typography-module__xdsH1___7oFBA ProductDetailsHeader-module__productTitle___Hce0B ProductDetailsHeaderProductTitle"]',
#         timeout=30
#     ).text

#     dados_jogo['span'] = scrape.get_element(
#         path='//div[@class="typography-module__xdsSubTitle1___N02-X ProductDetailsHeader-module__productInfoLine___W-v+p ProductInfoLinePublisherName"]',
#         timeout=30
#     ).text

#     dados_jogos.append(dados_jogo.copy())

#     browser.close()
#     scrape.switch_tab(tab=0, timeout=10)

# print(dados_jogos)

    


    

In [13]:
for iteration in links_na_pagina.keys():
    for link in links_na_pagina[f'{iteration}']:
        
        soup = dc.Soup(link)
        soup = soup.get_soup()

        dados_jogo['name'] = scrape.find_element(
            tag='h1', timeout=5, soup=soup,
            tag_class='typography-module__xdsH1___7oFBA ProductDetailsHeader-module__productTitle___Hce0B ProductDetailsHeaderProductTitle'
        ).get_text()

        dados_jogo['span'] = scrape.find_element(
            tag='div', timeout=1, soup=soup,
            tag_class='typography-module__xdsSubTitle1___N02-X ProductDetailsHeader-module__productInfoLine___W-v+p ProductInfoLinePublisherName'
        ).get_text()

        infos = scrape.find_element(
            tag='div', timeout=1, soup=soup,
            tag_class='ModuleRow-module__row___N1V3E Description-module__details___34Tnw'
        )


        infos = scrape.find_elements(
            tag='div', timeout=1, soup=infos,
            tag_class='typography-module__xdsBody2___RNdGY'
        )

        pub_dev_date = list()

        for info in infos:
            pub_dev_date.append(info.get_text())

        dados_jogo['pub_dev_date'] = pub_dev_date

        devices = scrape.find_element(
            tag='ul', timeout=1, soup=soup,
            tag_class='FeaturesList-module__wrapper___KIw42 commonStyles-module__featureListStyle___8SVho'
        )

        devices = scrape.find_elements(
            tag='li', timeout=1, soup=devices,
            tag_class='commonStyles-module__basicContainer___8Vx5e FeaturesList-module__item___J8r6P typography-module__xdsTag3___87dP9'
        )

        devices_list = list()

        for device in devices:
            devices_list.append(device.get_text())
        
        dados_jogo['devices'] = devices_list

        features = scrape.find_elements(
            tag='ul', timeout=1, soup=soup,
            tag_class='FeaturesList-module__wrapper___KIw42 commonStyles-module__featureListStyle___8SVho'
        )
        features = features[1]

        features = scrape.find_elements(
            tag='li', timeout=1, soup=features,
            tag_class='commonStyles-module__basicContainer___8Vx5e FeaturesList-module__item___J8r6P typography-module__xdsTag3___87dP9'
        )

        features_list = list()

        for feature in features:
            features_list.append(feature.get_text())
        
        dados_jogo['features'] = features_list
        dados_jogos.append(dados_jogo.copy())

In [ ]:
dados_jogos